In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.externals import joblib
from util_func.eval_function import auroc, cal_ks, auc, ks, fea_psi_calc
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing.data import OneHotEncoder
from sklearn import metrics
import sys
import math
from sklearn.metrics import roc_auc_score
import gc
from util_func import TCN_V3 as TCN
from util_func import self_attention as sf
from util_func.look_ahead import Lookahead
from util_func.LayerNormalization import LayerNormalization
import seaborn as sns
import random


load_model =  keras.models.load_model
Sequential = keras.models.Sequential
Dense = keras.layers.Dense
Activation = keras.layers.Activation
Dropout = keras.layers.Dropout
initializers = keras.initializers
regularizers = keras.regularizers
optimizers = keras.optimizers
Input = keras.layers.Input
add = keras.layers.add
Model = keras.models.Model
BatchNormalization = keras.layers.BatchNormalization 
EarlyStopping =  keras.callbacks.EarlyStopping
ModelCheckpoint = keras.callbacks.ModelCheckpoint
K = keras.backend
Concatenate = keras.layers.Concatenate
Reshape = keras.layers.Reshape
Flatten = keras.layers.Flatten
Lambda = keras.layers.Lambda

Using TensorFlow backend.


In [2]:
def vec_parse(fea_array, index):
    try:
        return fea_array[index]
    except:
        return 0.0
    


def seq_parse(s, fea_num, seq_length, dim=1):
    '''
    parse seq feature into matrix
    s:input sequcence feature string
    fea_num:number of features in a single seq node
    seq_length:max length of sequence
    dim:output dim, 1:flatten 2:matrix
    '''
    fea_str_list1=[]
    fea_array_all_node=[]
    
    if pd.isnull(s):
        return np.array([0.0]*fea_num*seq_length).reshape((1,-1))
    else:
        for ss in s.split('"')[1:-1]:
            if ss != ',':
                fea_str_list1.append(ss) # old from new 

        for ss in fea_str_list1: # every node 
            fea_str_list2 = ss.split(',')[1:]
            fea_array_per_node=[]
            for f in fea_str_list2:
                fea_array_per_node.append(f)

            if (len(fea_array_per_node) == fea_num):
                fea_array_all_node.append([float(n) for n in fea_array_per_node])

        if len(fea_array_all_node) < seq_length: # padding with zero
            for _ in range(seq_length-len(fea_array_all_node)):
                fea_array_all_node.insert(0, [0.0]*fea_num)

        fea_array_all_node = fea_array_all_node[-1*seq_length:]

        return np.array(fea_array_all_node).reshape((1,-1)) if dim==1 else np.array(fea_array_all_node)
    
def seq_feature_parse(data,name,fea_num,sequence_lengh):
    print(name,fea_num,sequence_lengh)
    data[name+'_seq_fea'] = data[name+'_seq_feature_string'].apply(lambda x: seq_parse(x, fea_num, sequence_lengh, 1))

    for i in range(sequence_lengh*fea_num):
        data[name+'_f' + str(i)] = data[name+'_seq_fea'].apply(lambda x:vec_parse(x[0], i))
    data = data.drop(columns=[name+'_seq_feature_string',name+'_seq_fea'])
    return data

def filter_feature(value, lower, upper):
    if value < lower:
        return lower
    elif value > upper:
        return upper
    else:
        return value

            


def range_filter(data,feature_range_path,cols):

    feature_range = open(feature_range_path, 'r')
    for line, col in zip(feature_range.readlines(), cols):
        lower, upper = float(line.strip().split(',')[0]), float(line.strip().split(',')[1])
        data[col] = data[col].apply(lambda x: filter_feature(x, lower, upper))

    return data



def data_split(data,split_col,split_date_1,split_date_2,rate_1,rate_2,old_sample_num):
    '''
    split_col:划分数据集的字段
    split_date_1：划分日期（上一版本的训练集时间），之前为老客，之后为次新客
    split_date_2：划分日期，之后为最新客，可以做为oot
    rate_1：老客/次新客划分进测试集的比例，一般设为0.2
    rate_2：最新客划分进测试集的比例，B卡V1.1为0.2，如果设为1，则产出split_date_2日期后一个完整的oot
    old_sample_num：老客采样数
    '''
    data_1_index = data[(data[split_col] < split_date_1) ].index.to_list()
    data_2_index = data[(data[split_col] >= split_date_1) & (data[split_col] < split_date_2)].index.to_list()
    data_3_index = data[(data[split_col] >= split_date_2) ].index.to_list() 
    random.shuffle(data_1_index)
    random.shuffle(data_2_index)
    random.shuffle(data_3_index)
   
    test_1_index = data_1_index[:round(float(rate_1)*len(data_1_index))]  
    test_2_index = data_2_index[:round(float(rate_1)*len(data_2_index))] 
    test_3_index = data_3_index[:round(float(rate_2)*len(data_3_index))]

    train_1_index = data_1_index[round(float(rate_1)*len(data_1_index)):][:old_sample_num]
    train_2_index = data_2_index[round(float(rate_1)*len(data_2_index)):] 
    train_3_index = data_3_index[round(float(rate_2)*len(data_3_index)):]
    
    test_index = test_1_index + test_2_index + test_3_index
    train_index = train_1_index+ train_2_index + train_3_index
    return train_index,test_index

def data_split_2(data,split_col,split_date_1,rate_1):
    '''
    split_col:划分数据集的字段
    split_date_1：划分日期，之后为最新客，可以做为oot
    rate_1:训练集同时间段的验证集 比例
    '''
    print(split_col,split_date_1,rate_1)
    data_1_index = data[(data[split_col] < split_date_1) ].index.to_list()
    data_2_index = data[(data[split_col] >= split_date_1) ].index.to_list() 
    random.shuffle(data_1_index)
    
    train_index = data_1_index[round(float(rate_1)*len(data_1_index)):]
    valid_index = data_1_index[:round(float(rate_1)*len(data_1_index))]
    test_index = data_2_index
    return train_index,valid_index,test_index



def get_scaler_online_file(scaler_model_path,scaler_online_file_path):
    zs_base = joblib.load(scaler_model_path)
    f=open(scaler_online_file_path, 'w')
    for m,v in zip(zs_base.mean_, zs_base.var_):
        f.write(str(m) + ',' + str(v) + '\n')
    f.close()


In [3]:
def fea_range_filter(data, sigma_threshold):
    cols = data.columns
    for col in cols:
        mean = data[col].mean()
        std = data[col].std()
        data[col] = data[col].apply(lambda x: filter_feature(x, mean - sigma_threshold*std, mean + sigma_threshold*std))
    return data



In [4]:

LOAN_SEQ_LENGTH=10
LOAN_FEATURE_NUM=26
REPAY_SEQ_LENGTH=15
REPAY_FEATURE_NUM=17
EVENT_ACTION_SEQ_LENGTH=30
EVENT_ACTION_FEATURE_NUM=2
EVENT_CHAIN_SEQ_LENGTH=20
EVENT_CHAIN_FEATURE_NUM=55

STAT_FEATURE_NUM = 197

LOAN_FEATURE_INDEX_START=0+7
LOAN_FEATURE_INDEX_END=260+7
REPAY_FEATURE_INDEX_START=260+7
REPAY_FEATURE_INDEX_END=515+7
EVENT_ACTION_FEATURE_INDEX_START=515+7
EVENT_ACTION_FEATURE_INDEX_END=575+7
EVENT_CHAIN_FEATURE_INDEX_START=575+7
EVENT_CHAIN_FEATURE_INDEX_END=1675+7

DATE_1 = '2020-08-08'
RATE_1 = 0.1

In [5]:
data_dir = '/home/odin/chengbixiao/git_code/User_loan_demand/user_loan_demand/offline/data/'
seq_data_path = data_dir + 'feature_ori_file/dsd_user_loan_demand_loan_repay_event_seq_strings_offline_old.csv'

train_info_path = data_dir + 'feature_model_file/train_info.csv'
test1_info_path = data_dir + 'feature_model_file/test1_info.csv'
test_info_path = data_dir + 'feature_model_file/test_info.csv'

train_label_path = data_dir + 'feature_model_file/train_label.npy'
test1_label_path = data_dir + 'feature_model_file/test1_label.npy'
test_label_path = data_dir + 'feature_model_file/test_label.npy'

loan_scaler_model_path = data_dir + 'feature_process_file/loan_scaler_model.model'
repay_scaler_model_path =  data_dir + 'feature_process_file/repay_scaler_model.model'
event1_scaler_model_path = data_dir + 'feature_process_file/event1_scaler_model.model'
event2_scaler_model_path =  data_dir + 'feature_process_file/event2_scaler_model.model'


train_loan_seq_matrix_path = data_dir + 'feature_model_file/train_loan_seq_matrix.npy'
train_repay_seq_matrix_path = data_dir + 'feature_model_file/train_repay_seq_matrix.npy'
train_event1_seq_matrix_path = data_dir + 'feature_model_file/train_event1_seq_matrix.npy'
train_event2_seq_matrix_path = data_dir + 'feature_model_file/train_event2_seq_matrix.npy'

test1_loan_seq_matrix_path = data_dir + 'feature_model_file/test1_loan_seq_matrix.npy'
test1_repay_seq_matrix_path = data_dir + 'feature_model_file/test1_repay_seq_matrix.npy'
test1_event1_seq_matrix_path = data_dir + 'feature_model_file/test1_event1_seq_matrix.npy'
test1_event2_seq_matrix_path = data_dir + 'feature_model_file/test1_event2_seq_matrix.npy'

test_loan_seq_matrix_path = data_dir + 'feature_model_file/test_loan_seq_matrix.npy'
test_repay_seq_matrix_path = data_dir + 'feature_model_file/test_repay_seq_matrix.npy'
test_event1_seq_matrix_path = data_dir + 'feature_model_file/test_event1_seq_matrix.npy'
test_event2_seq_matrix_path = data_dir + 'feature_model_file/test_event2_seq_matrix.npy'


# stat_data_path = data_dir + 'feature_ori_file/feat_all.tsv'
stat_data_path = '/home/odin/lijunwei/yiyuan/feat_100w.csv'
stat_scaler_model_path =  data_dir + 'feature_process_file/stat_scaler_model.model'

stat_feature_train_norm_path = data_dir + 'feature_model_file/stat_feature_train.npy'
stat_feature_test1_norm_path = data_dir + 'feature_model_file/stat_feature_test1.npy'
stat_feature_test_norm_path = data_dir + 'feature_model_file/stat_feature_test.npy'

# load seq data

In [6]:
# 1.load seq data
print("seq_data_path = " ,seq_data_path)
seq_data = pd.read_csv(seq_data_path,sep='\t',nrows=1000000)
seq_data.columns = [x.split('.')[1] for x in seq_data.columns]
print("SEQ DATA load FINISH !")

seq_data_path =  /home/odin/chengbixiao/git_code/User_loan_demand/user_loan_demand/offline/data/feature_ori_file/dsd_user_loan_demand_loan_repay_event_seq_strings_offline_old.csv
SEQ DATA load FINISH !


In [7]:
seq_data.columns

Index(['uid', 'tel', 'funds_channel_id', 'server_date',
       'dsd_market_biopen_tag', 'dsd_loan_confirm_button_ck_tag', 'label',
       'loan_seq_feature_string', 'repay_seq_feature_string',
       'event1_action_seq_feature_string', 'event2_chain_seq_feature_string'],
      dtype='object')

In [8]:
seq_data.shape

(1000000, 11)

# 1.seq parse

In [9]:
seq_data = seq_feature_parse(seq_data,'loan',LOAN_FEATURE_NUM,LOAN_SEQ_LENGTH)
print("LOAN FEATURE FINISH !")

loan 26 10
LOAN FEATURE FINISH !


In [10]:
seq_data = seq_feature_parse(seq_data,'repay',REPAY_FEATURE_NUM,REPAY_SEQ_LENGTH)
print("REPAY FEATURE FINISH !")

repay 17 15
REPAY FEATURE FINISH !


In [11]:
seq_data = seq_feature_parse(seq_data,'event1_action',EVENT_ACTION_FEATURE_NUM,EVENT_ACTION_SEQ_LENGTH)
print("EVENT_ACTION FEATURE FINISH !")

event1_action 2 30
EVENT_ACTION FEATURE FINISH !


In [12]:
gc.collect()

281

In [13]:

def seq_parse(s, fea_num, seq_length, dim=1):
    '''
    parse seq feature into matrix
    s:input sequcence feature string
    fea_num:number of features in a single seq node
    seq_length:max length of sequence
    dim:output dim, 1:flatten 2:matrix
    '''
    fea_str_list1=[]
    fea_array_all_node=[]
    
    if pd.isnull(s):
        return np.array([0.0]*fea_num*seq_length).reshape((1,-1))
    else:
        for ss in s.split('"')[1:-1]:
            if ss != ',':
                fea_str_list1.append(ss) # old from new 
        
        for ss in fea_str_list1: # every node 
            fea_str_list2 = ss.split(',')
            fea_array_per_node=[]
            for f in fea_str_list2:
                fea_array_per_node.append(f)
            if (len(fea_array_per_node) == fea_num):
                fea_array_all_node.append([float(n) for n in fea_array_per_node])
                
        if len(fea_array_all_node) < seq_length: # padding with zero
            for _ in range(seq_length-len(fea_array_all_node)):
                fea_array_all_node.insert(0, [0.0]*fea_num)

        fea_array_all_node = fea_array_all_node[-1*seq_length:]

        return np.array(fea_array_all_node).reshape((1,-1)) if dim==1 else np.array(fea_array_all_node)

In [14]:
seq_data = seq_feature_parse(seq_data,'event2_chain',EVENT_CHAIN_FEATURE_NUM,EVENT_CHAIN_SEQ_LENGTH)
print("EVENT_CHAIN FEATURE FINISH !")

event2_chain 55 20
EVENT_CHAIN FEATURE FINISH !


In [15]:
pd.set_option('display.max_columns', None)
seq_data.head()

uid          tel  funds_channel_id server_date  dsd_market_biopen_tag  \
0  10054929  18282052338              3038  2020-07-26                      1   
1  10670317  13424191148              3038  2020-07-30                      1   
2  11276703  15889775882              3038  2020-07-30                      1   
3  11438051  18757923981              3018  2020-07-18                      1   
4  11906089  13358008179              3038  2020-07-28                      1   

   dsd_loan_confirm_button_ck_tag  label  loan_f0  loan_f1  loan_f2  loan_f3  \
0                               0      0      0.0      0.0      0.0      0.0   
1                               1      1      0.0      0.0      0.0      0.0   
2                               0      0      0.0      0.0      0.0      0.0   
3                               0      0      1.0      0.0     12.0    972.0   
4                               0      0      0.0      0.0      0.0      0.0   

   loan_f4  loan_f5  loan_f6  loan_f7  loan_f8  loan_f9  loan_f10  loan_f11  \
0      0.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0   
1      0.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0   
2      0.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0   
3      1.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0   
4      0.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0   

   loan_f12  loan_f13  loan_f14  loan_f15  loan_f16  loan_f17  loan_f18  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   loan_f19  loan_f20  loan_f21  loan_f22  loan_f23  loan_f24  loan_f25  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   loan_f26  loan_f27  loan_f28  loan_f29  loan_f30  loan_f31  loan_f32  \
0       0.0    0.0000       0.0       0.0       0.0       0.0       0.0   
1       0.0    0.0000       0.0       0.0       0.0       0.0       0.0   
2       0.0    0.0000       0.0       0.0       0.0       0.0       0.0   
3       0.0    0.5313      12.0     972.0       1.0       0.0       0.0   
4       0.0    0.0000       0.0       0.0       0.0       0.0       0.0   

   loan_f33  loan_f34  loan_f35  loan_f36  loan_f37  loan_f38  loan_f39  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   loan_f40  loan_f41  loan_f42  loan_f43  loan_f44  loan_f45  loan_f46  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0     195.0     195.0       0.0       1.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   loan_f47  loan_f48  loan_f49  loan_f50  loan_f51  loan_f52  loan_f53  \
0       0.0       0.0       0.0    0.0000    0.0000       0.0    0.0000   
1       0.0       0.0       0.0    0.0000    0.0000       0.0    0.0000   
2       0.0       0.0       0.0    0.0000    0.0000       0.0    0.0000   
3       1.

In [16]:
7 + 260 + 255 + 60 + 1100

1682

In [17]:
seq_data.shape

(1000000, 1682)

# 2.feature range

In [ ]:
seq_data

In [18]:
# 2. range
seq_data.iloc[:, LOAN_FEATURE_INDEX_START:LOAN_FEATURE_INDEX_END]=fea_range_filter(seq_data.iloc[:, LOAN_FEATURE_INDEX_START:LOAN_FEATURE_INDEX_END], 6)

In [19]:
seq_data.iloc[:, REPAY_FEATURE_INDEX_START:REPAY_FEATURE_INDEX_END]=fea_range_filter(seq_data.iloc[:, REPAY_FEATURE_INDEX_START:REPAY_FEATURE_INDEX_END], 6)

In [20]:
seq_data.iloc[:, EVENT_ACTION_FEATURE_INDEX_START:EVENT_ACTION_FEATURE_INDEX_END]=fea_range_filter(seq_data.iloc[:, EVENT_ACTION_FEATURE_INDEX_START:EVENT_ACTION_FEATURE_INDEX_END], 6)


In [21]:
seq_data.iloc[:, EVENT_CHAIN_FEATURE_INDEX_START:EVENT_CHAIN_FEATURE_INDEX_END]=fea_range_filter(seq_data.iloc[:, EVENT_CHAIN_FEATURE_INDEX_START:EVENT_CHAIN_FEATURE_INDEX_END], 6)


In [22]:
seq_data.head()

uid          tel  funds_channel_id server_date  dsd_market_biopen_tag  \
0  10054929  18282052338              3038  2020-07-26                      1   
1  10670317  13424191148              3038  2020-07-30                      1   
2  11276703  15889775882              3038  2020-07-30                      1   
3  11438051  18757923981              3018  2020-07-18                      1   
4  11906089  13358008179              3038  2020-07-28                      1   

   dsd_loan_confirm_button_ck_tag  label  loan_f0  loan_f1  loan_f2  loan_f3  \
0                               0      0      0.0      0.0      0.0      0.0   
1                               1      1      0.0      0.0      0.0      0.0   
2                               0      0      0.0      0.0      0.0      0.0   
3                               0      0      1.0      0.0     12.0    972.0   
4                               0      0      0.0      0.0      0.0      0.0   

   loan_f4  loan_f5  loan_f6  loan_f7  loan_f8  loan_f9  loan_f10  loan_f11  \
0      0.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0   
1      0.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0   
2      0.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0   
3      1.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0   
4      0.0      0.0      0.0      0.0      0.0      0.0       0.0       0.0   

   loan_f12  loan_f13  loan_f14  loan_f15  loan_f16  loan_f17  loan_f18  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   loan_f19  loan_f20  loan_f21  loan_f22  loan_f23  loan_f24  loan_f25  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   loan_f26  loan_f27  loan_f28  loan_f29  loan_f30  loan_f31  loan_f32  \
0       0.0    0.0000       0.0       0.0       0.0       0.0       0.0   
1       0.0    0.0000       0.0       0.0       0.0       0.0       0.0   
2       0.0    0.0000       0.0       0.0       0.0       0.0       0.0   
3       0.0    0.5313      12.0     972.0       1.0       0.0       0.0   
4       0.0    0.0000       0.0       0.0       0.0       0.0       0.0   

   loan_f33  loan_f34  loan_f35  loan_f36  loan_f37  loan_f38  loan_f39  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   loan_f40  loan_f41  loan_f42  loan_f43  loan_f44  loan_f45  loan_f46  \
0       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
1       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
2       0.0       0.0       0.0       0.0       0.0       0.0       0.0   
3       0.0       0.0     195.0     195.0       0.0       1.0       0.0   
4       0.0       0.0       0.0       0.0       0.0       0.0       0.0   

   loan_f47  loan_f48  loan_f49  loan_f50  loan_f51  loan_f52  loan_f53  \
0       0.0       0.0       0.0    0.0000    0.0000       0.0    0.0000   
1       0.0       0.0       0.0    0.0000    0.0000       0.0    0.0000   
2       0.0       0.0       0.0    0.0000    0.0000       0.0    0.0000   
3       1.

# 3.split data

In [23]:
# 3.
train_index,test1_index,test_index = data_split_2(seq_data,'server_date',DATE_1,RATE_1)
print("DATA SPLIT FINASH ! TRAIN AND TEST SIZE =",len(train_index),",",len(test1_index),",",len(test_index))

server_date 2020-08-08 0.1
DATA SPLIT FINASH ! TRAIN AND TEST SIZE = 781990 , 86888 , 131122


In [61]:
seq_data.columns[:10]

Index(['uid', 'tel', 'funds_channel_id', 'server_date',
       'dsd_market_biopen_tag', 'dsd_loan_confirm_button_ck_tag', 'label',
       'loan_f0', 'loan_f1', 'loan_f2'],
      dtype='object')

In [24]:
# save info and label data
sub_cols = ['uid','server_date','label']

train_info = seq_data.loc[train_index, sub_cols]
test1_info = seq_data.loc[test1_index, sub_cols]
test_info = seq_data.loc[test_index, sub_cols]
train_info.to_csv(train_info_path)
test1_info.to_csv(test1_info_path)
test_info.to_csv(test_info_path)

train_label = seq_data.loc[train_index, 'label']
test1_label = seq_data.loc[test1_index, 'label']
test_label = seq_data.loc[test_index, 'label']
np.save(train_label_path,train_label)
np.save(test1_label_path,test1_label)
np.save(test_label_path,test_label)

print("SAVE FINASH ! TRAIN AND TEST SIZE =",len(train_label),train_info.shape,",",len(test1_label),test1_label.shape,",",len(test_label),test_label.shape)

SAVE FINASH ! TRAIN AND TEST SIZE = 781990 (781990, 3) , 86888 (86888,) , 131122 (131122,)


# 4.scaler

In [25]:
 ###按照train归一化 fit
def scaler_process_fit(data,train_index,loan_scaler_model_path,repay_scaler_model_path,event1_scaler_model_path,event2_scaler_model_path):
    scaler_loan = preprocessing.StandardScaler()
    scaler_loan.fit(data.iloc[train_index, LOAN_FEATURE_INDEX_START:LOAN_FEATURE_INDEX_END])
    joblib.dump(scaler_loan, filename=loan_scaler_model_path)

    scaler_repay = preprocessing.StandardScaler()
    scaler_repay.fit(data.iloc[train_index, REPAY_FEATURE_INDEX_START:REPAY_FEATURE_INDEX_END])
    joblib.dump(scaler_repay, filename=repay_scaler_model_path)
    
    scaler_event1 = preprocessing.StandardScaler()
    scaler_event1.fit(data.iloc[train_index, EVENT_ACTION_FEATURE_INDEX_START:EVENT_ACTION_FEATURE_INDEX_END])
    joblib.dump(scaler_event1, filename=event1_scaler_model_path)
 
    scaler_event2 = preprocessing.StandardScaler()
    scaler_event2.fit(data.iloc[train_index, EVENT_CHAIN_FEATURE_INDEX_START:EVENT_CHAIN_FEATURE_INDEX_END])
    joblib.dump(scaler_event2, filename=event2_scaler_model_path)

    
def scaler_process_transform(data,index_t,loan_scaler_model_path,repay_scaler_model_path,event1_scaler_model_path,event2_scaler_model_path):
    print('Loan seq feature Normalization...')
    scaler_loan = joblib.load(loan_scaler_model_path)
    data.iloc[index_t, LOAN_FEATURE_INDEX_START:LOAN_FEATURE_INDEX_END] = scaler_loan.transform(data.iloc[index_t, LOAN_FEATURE_INDEX_START:LOAN_FEATURE_INDEX_END])        
    del scaler_loan
    gc.collect()
    
    print('Repay seq feature Normalization...')
    scaler_repay = joblib.load(repay_scaler_model_path)
    data.iloc[index_t, REPAY_FEATURE_INDEX_START:REPAY_FEATURE_INDEX_END] = scaler_repay.transform(data.iloc[index_t, REPAY_FEATURE_INDEX_START:REPAY_FEATURE_INDEX_END])        
    del scaler_repay
    gc.collect()

    print('event1 seq feature Normalization...')
    scaler_event1 = joblib.load(event1_scaler_model_path)
    data.iloc[index_t, EVENT_ACTION_FEATURE_INDEX_START:EVENT_ACTION_FEATURE_INDEX_END] = scaler_event1.transform(data.iloc[index_t, EVENT_ACTION_FEATURE_INDEX_START:EVENT_ACTION_FEATURE_INDEX_END])        
    del scaler_event1
    gc.collect()
    
    print('event2 seq feature Normalization...')
    scaler_event2 = joblib.load(event2_scaler_model_path)
    data.iloc[index_t, EVENT_CHAIN_FEATURE_INDEX_START:EVENT_CHAIN_FEATURE_INDEX_END] = scaler_event2.transform(data.iloc[index_t, EVENT_CHAIN_FEATURE_INDEX_START:EVENT_CHAIN_FEATURE_INDEX_END])        
    del scaler_event2
    gc.collect()
    

In [26]:
scaler_process_fit(seq_data,train_index,loan_scaler_model_path,repay_scaler_model_path,event1_scaler_model_path,event2_scaler_model_path)
gc.collect()


85

In [27]:
scaler_process_transform(seq_data,train_index,loan_scaler_model_path,repay_scaler_model_path,event1_scaler_model_path,event2_scaler_model_path)
gc.collect()


Loan seq feature Normalization...
Repay seq feature Normalization...
event1 seq feature Normalization...
event2 seq feature Normalization...


0

In [28]:
scaler_process_transform(seq_data,test1_index,loan_scaler_model_path,repay_scaler_model_path,event1_scaler_model_path,event2_scaler_model_path)
gc.collect()


Loan seq feature Normalization...
Repay seq feature Normalization...
event1 seq feature Normalization...
event2 seq feature Normalization...


0

In [29]:
scaler_process_transform(seq_data,test_index,loan_scaler_model_path,repay_scaler_model_path,event1_scaler_model_path,event2_scaler_model_path)
gc.collect()
print("SCALER PROCESS FINISH !")

Loan seq feature Normalization...
Repay seq feature Normalization...
event1 seq feature Normalization...
event2 seq feature Normalization...
SCALER PROCESS FINISH !


In [30]:
# into 2D matrix

loan_seq_matrix = np.array(seq_data.iloc[:, LOAN_FEATURE_INDEX_START:LOAN_FEATURE_INDEX_END]).reshape((-1,LOAN_SEQ_LENGTH, LOAN_FEATURE_NUM))
repay_seq_matrix = np.array(seq_data.iloc[:, REPAY_FEATURE_INDEX_START:REPAY_FEATURE_INDEX_END]).reshape((-1,REPAY_SEQ_LENGTH, REPAY_FEATURE_NUM))
event1_seq_matrix = np.array(seq_data.iloc[:, EVENT_ACTION_FEATURE_INDEX_START:EVENT_ACTION_FEATURE_INDEX_END]).reshape((-1,EVENT_ACTION_SEQ_LENGTH, EVENT_ACTION_FEATURE_NUM))
event2_seq_matrix = np.array(seq_data.iloc[:, EVENT_CHAIN_FEATURE_INDEX_START:EVENT_CHAIN_FEATURE_INDEX_END]).reshape((-1,EVENT_CHAIN_SEQ_LENGTH, EVENT_CHAIN_FEATURE_NUM))
print("GENERATE SEQUENCE FEATURE MATRIX FINISH!")


train_loan_seq_matrix = loan_seq_matrix[train_index, :, :]
train_repay_seq_matrix = repay_seq_matrix[train_index, :, :]
train_event1_seq_matrix = event1_seq_matrix[train_index, :, :]
train_event2_seq_matrix = event2_seq_matrix[train_index, :, :]

test1_loan_seq_matrix = loan_seq_matrix[test1_index, :, :]
test1_repay_seq_matrix = repay_seq_matrix[test1_index, :, :]
test1_event1_seq_matrix = event1_seq_matrix[test1_index, :, :]
test1_event2_seq_matrix = event2_seq_matrix[test1_index, :, :]

test_loan_seq_matrix = loan_seq_matrix[test_index, :, :]
test_repay_seq_matrix = repay_seq_matrix[test_index, :, :]
test_event1_seq_matrix = event1_seq_matrix[test_index, :, :]
test_event2_seq_matrix = event2_seq_matrix[test_index, :, :]

print("SPLIT INTO TRAIN AND TEST DATASET FINISH!")

# 7. save

np.save(train_loan_seq_matrix_path,train_loan_seq_matrix)
np.save(train_repay_seq_matrix_path,train_repay_seq_matrix)
np.save(train_event1_seq_matrix_path,train_event1_seq_matrix)
np.save(train_event2_seq_matrix_path,train_event2_seq_matrix)

np.save(test_loan_seq_matrix_path,test_loan_seq_matrix)
np.save(test_repay_seq_matrix_path,test_repay_seq_matrix)
np.save(test_event1_seq_matrix_path,test_event1_seq_matrix)
np.save(test_event2_seq_matrix_path,test_event2_seq_matrix)
print("SAVE TRAIN AND TEST NPY FINISH!")

GENERATE SEQUENCE FEATURE MATRIX FINISH!
SPLIT INTO TRAIN AND TEST DATASET FINISH!
SAVE TRAIN AND TEST NPY FINISH!


In [31]:
np.save(test1_loan_seq_matrix_path,test1_loan_seq_matrix)
np.save(test1_repay_seq_matrix_path,test1_repay_seq_matrix)
np.save(test1_event1_seq_matrix_path,test1_event1_seq_matrix)
np.save(test1_event2_seq_matrix_path,test1_event2_seq_matrix)

In [32]:
gc.collect()

44

In [1]:
seq_data['label'].value_counts()

NameError: name 'seq_data' is not defined

# stat data 

In [6]:
# 1.load baseline data
stat_data = pd.read_csv(stat_data_path,sep=',')
# stat_data.columns = [x.split('.')[1] for x in stat_data.columns]
stat_data.iloc[:,-1*STAT_FEATURE_NUM:] = stat_data.iloc[:,-1*STAT_FEATURE_NUM:].fillna(0.0)
print("BASE DATA LOAD FINISH !")

BASE DATA LOAD FINISH !


In [7]:
stat_data.shape

(1000740, 200)

In [8]:
del stat_data['last_drip_funds_channel_id']
del stat_data['last_credit_identity_card_city_id']
del stat_data['last_credit_mobile_city_id']

In [9]:
stat_data.shape

(1000740, 197)

In [11]:
STAT_FEATURE_NUM=194

In [12]:
# 2. range
stat_data.iloc[:, -1*STAT_FEATURE_NUM:]=fea_range_filter(stat_data.iloc[:, -1*STAT_FEATURE_NUM:], 6)

In [13]:
stat_data.head()

,uid,server_date,label,current_drip_loan_credit_line_use_rate,current_drip_loan_credit_line_available,last_credit_success_end_time,his_drip_max_credit_line,first_drip_apply_loan_pass_time,last_credit_success_start_time,last_drip_apply_loan_time,...,last_3d_market_loan_list_sw_cnt,last_7d_loan_page_show_cnt,last_7d_loan_apply_submit_api_done_cnt,ratio_3,last_7d_market_setup_sw_cnt,is_last_1d_market_biopen,last_7d_market_biopen_cnt,ratio_7,is_last_1d_loan_confirm_button_ck,last_1m_input_pay_password_popup_cnt
0,299067330360215,2020-07-29,0,0.694210,1528.95,-265.0,5000.0,100.0,99.0,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,281475087995328,2020-07-17,0,0.554589,5211.31,-29021.0,11700.0,436.0,436.0,436.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,284111221556771,2020-07-25,1,0.250011,2099.97,210.0,2800.0,301.0,301.0,301.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,299067257604563,2020-07-18,0,0.329391,32189.22,-106.0,48000.0,578.0,578.0,578.0,...,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
4,285016812694872,2020-07-22,0,0.090909,10000.00,-102.0,11000.0,585.0,585.0,585.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [14]:
train_info = pd.read_csv(train_info_path)
test1_info = pd.read_csv(test1_info_path)
test_info = pd.read_csv(test_info_path)

In [15]:
train_info.shape,test1_info.shape,test_info.shape

((781990, 4), (86888, 4), (131122, 4))

In [16]:
seq_info = pd.concat([train_info,test1_info,test_info],axis=0)
seq_info = seq_info.reset_index(drop=True)
train_index = seq_info.index.to_list()[:train_info.shape[0]]
test1_index = seq_info.index.to_list()[train_info.shape[0]:train_info.shape[0]+test1_info.shape[0]]
test_index = seq_info.index.to_list()[train_info.shape[0]+test1_info.shape[0]:]

In [17]:
seq_info.shape,len(train_index),len(test1_index),len(test_index)

((1000000, 4), 781990, 86888, 131122)

In [18]:
stat_data = stat_data.drop_duplicates(subset=['uid', 'server_date'], keep='first' )

In [19]:
del stat_data['label']

In [20]:
# 3.seq merge get train and test index
# seq_info = seq_data[seq_data.columns[:7]]
# del seq_data
baseline_feature = pd.merge(seq_info, stat_data, on=['uid', 'server_date'],how='left')
print("GET TRAIN AND TEST INDEX FINISH!")

GET TRAIN AND TEST INDEX FINISH!


In [21]:
baseline_feature.shape

(1000000, 198)

In [22]:
seq_info.head()

,Unnamed: 0,uid,server_date,label
0,561549,299067330360215,2020-07-29,0
1,433701,281475087995328,2020-07-17,0
2,467531,284111221556771,2020-07-25,1
3,487451,299067257604563,2020-07-18,0
4,693473,285016812694872,2020-07-22,0


In [23]:
baseline_feature.head()

,Unnamed: 0,uid,server_date,label,current_drip_loan_credit_line_use_rate,current_drip_loan_credit_line_available,last_credit_success_end_time,his_drip_max_credit_line,first_drip_apply_loan_pass_time,last_credit_success_start_time,...,last_3d_market_loan_list_sw_cnt,last_7d_loan_page_show_cnt,last_7d_loan_apply_submit_api_done_cnt,ratio_3,last_7d_market_setup_sw_cnt,is_last_1d_market_biopen,last_7d_market_biopen_cnt,ratio_7,is_last_1d_loan_confirm_button_ck,last_1m_input_pay_password_popup_cnt
0,561549,299067330360215,2020-07-29,0,0.694210,1528.95,-265.0,5000.0,100.0,99.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,433701,281475087995328,2020-07-17,0,0.554589,5211.31,-29021.0,11700.0,436.0,436.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,467531,284111221556771,2020-07-25,1,0.250011,2099.97,210.0,2800.0,301.0,301.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,487451,299067257604563,2020-07-18,0,0.329391,32189.22,-106.0,48000.0,578.0,578.0,...,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
4,693473,285016812694872,2020-07-22,0,0.090909,10000.00,-102.0,11000.0,585.0,585.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [25]:
scaler_stat = preprocessing.StandardScaler()
scaler_stat.fit(baseline_feature.iloc[train_index, -1*STAT_FEATURE_NUM:])
joblib.dump(scaler_stat, filename=stat_scaler_model_path)
print('baseline feature Normalization...')
scaler_stat = joblib.load(stat_scaler_model_path)
baseline_feature.iloc[:, -1*STAT_FEATURE_NUM:] = scaler_stat.transform(baseline_feature.iloc[:, -1*STAT_FEATURE_NUM:])        
del scaler_stat
gc.collect()

baseline feature Normalization...


36

In [26]:
print("SCALER PROCESS FINISH !")

# 5.split into train and test 
stat_feature_train = baseline_feature.iloc[train_index,  -1*STAT_FEATURE_NUM:]
stat_feature_test1 = baseline_feature.iloc[test1_index,  -1*STAT_FEATURE_NUM:]
stat_feature_test = baseline_feature.iloc[test_index,  -1*STAT_FEATURE_NUM:]
print("SPLIT INTO TRAIN AND TEST FINISH !")

#  6.save
np.save(stat_feature_train_norm_path,stat_feature_train)
np.save(stat_feature_test1_norm_path,stat_feature_test1)
np.save(stat_feature_test_norm_path,stat_feature_test)
print("SAVE FINISH !")

SCALER PROCESS FINISH !
SPLIT INTO TRAIN AND TEST FINISH !
SAVE FINISH !


In [27]:
stat_feature_train.shape

(781990, 194)

In [28]:

def print_result(y_score, y_test, thr=0.5):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_score, pos_label = 1)
    print('auc:{}'.format(metrics.auc(fpr, tpr)))

def recall(y_pred, dtrain):
    y_true = dtrain.get_label()
    recall = recall_score(y_true,np.round(y_pred))
    return 'recall', recall

def ks(y_true, y_pred):
    fpr,tpr,thresholds= metrics.roc_curve(y_true,y_pred)
    return max(tpr-fpr)

def print_feature_importance(feature, bst, method='weight'):
    top_fea = []
    top_col=[]
    n=0
    if method == 'weight':
        fs = bst.get_fscore()
        all_features = [fs.get(f, 0.) for f in bst.feature_names]
        all_features = np.array(all_features, dtype=np.float32)
        feature_importance = all_features / all_features.sum()
        feature_combine = sorted(zip(feature, feature_importance), key=lambda x: x[1], reverse=True)
        for ff, imp in feature_combine:
            if imp > 0:
#                 print(ff, imp)
                top_fea.append([ff,imp])
                top_col.append(ff)
            n += 1
    elif method == 'gain':
        importance = bst.get_score(importance_type='gain');
        importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True);
        for ff, imp in importance:
            print(ff, imp)
            top_fea.append([ff,imp])
            n +=1 
#     print(n)
    return top_fea, top_col

###计算单变量psi
def fea_psi_calc(actual,predict,bins=10):
    '''
    功能: 计算连续变量和离散变量的PSI值
    输入值:
    actual: 一维数组或series，代表训练集中的变量
    predict: 一维数组或series，代表测试集中的变量
    bins: 违约率段划分个数
    输出值: 
    字典，键值关系为{'psi': PSI值，'psi_fig': 实际和预期占比分布曲线}
    '''  
    psi_dict = {}
    psi_cut = []
    actual_bins = []
    predict_bins = []
    actual_len = len(actual)
    predict_len = len(predict)
    #print(actual.isnull().any())
    if actual.isnull().any() == True:
        #print("has null")
        bins = bins-1
        actual_cnt = actual.isna().sum()
        predict_cnt = predict.isna().sum()
        actual_pct = (actual_cnt+0.0) / actual_len
        predict_pct = (predict_cnt+0.0) / predict_len
        psi = (predict_pct-actual_pct) * math.log((predict_pct+0.00000001)/actual_pct)
        psi_cut.append(psi)
        actual_bins.append(actual_pct)
        predict_bins.append(predict_pct)

    actual_temp = actual.dropna()
    predict_temp = predict.dropna()
    if len(actual_temp) > 0: 
        actual = np.sort(actual_temp)
        actual_distinct = np.sort(list(set(actual)))
        predict = np.sort(predict_temp)
        predict_distinct = np.sort(list(set(predict)))
        actual_distinct_len = len(actual_distinct)
        predict_distinct_len = len(predict_distinct)
        actual_min = actual.min()
        actual_max = actual.max()
        cuts = []
        binlen = (actual_max-actual_min) / bins
        if (actual_distinct_len<bins):
            for i in actual_distinct:
                cuts.append(i)
            for i in range(2, (actual_distinct_len+2)):
                if i == actual_distinct_len+1:
                    lowercut = cuts[i-2]
                    uppercut = float('Inf')
                else:
                    lowercut = cuts[i-2]
                    uppercut = cuts[i-1]
                actual_cnt = ((actual >= lowercut) & (actual < uppercut)).sum()+1
                predict_cnt = ((predict >= lowercut) & (predict < uppercut)).sum()+1
                actual_pct = (actual_cnt+0.0) / actual_len
                predict_pct = (predict_cnt+0.0) / predict_len
                psi_cut.append((predict_pct-actual_pct) * math.log((predict_pct+0.00000001)/actual_pct))
                actual_bins.append(actual_pct)
                predict_bins.append(predict_pct)
        else:
            for i in range(1, bins):
                cuts.append(actual_min+i*binlen)
            for i in range(1, (bins+1)):
                if i == 1:
                    lowercut = float('-Inf')
                    uppercut = cuts[i-1]
                elif i == bins:
                    lowercut = cuts[i-2]
                    uppercut = float('Inf')
                else:
                    lowercut = cuts[i-2]
                    uppercut = cuts[i-1]
                actual_cnt = ((actual >= lowercut) & (actual < uppercut)).sum()+1
                predict_cnt = ((predict >= lowercut) & (predict < uppercut)).sum()+1
                actual_pct = (actual_cnt+0.0) / actual_len
                predict_pct = (predict_cnt+0.0) / predict_len
                psi_cut.append((predict_pct-actual_pct) * math.log((predict_pct+0.00000001)/actual_pct))
                actual_bins.append(actual_pct)
                predict_bins.append(predict_pct)
    psi = sum(psi_cut)
    nbins = len(actual_bins)
    xlab = np.arange(1, nbins+1)
    psi_dict['psi'] = psi
    return psi_dict

def score_psi_calc(actual,predict,bins=10):
    '''
    功能: 计算连续变量和离散变量的PSI值
    输入值:
    actual: 一维数组或series，代表训练集中的变量
    predict: 一维数组或series，代表测试集中的变量
    bins: 违约率段划分个数
    输出值: 
    字典，键值关系为{'psi': PSI值，'psi_fig': 实际和预期占比分布曲线}
    '''
    psi_dict = {}        
    actual = np.sort(actual)
    actual_distinct = np.sort(list(set(actual)))
    predict = np.sort(predict)
    predict_distinct = np.sort(list(set(predict)))
    actual_len = len(actual)
    actual_distinct_len = len(actual_distinct)
    predict_len = len(predict)
    predict_distinct_len = len(predict_distinct)
    psi_cut = []
    actual_bins = []
    predict_bins = []
    actual_min = actual.min()
    actual_max = actual.max()
    cuts = []
    binlen = (actual_max-actual_min) / bins
    if (actual_distinct_len<bins):
        for i in actual_distinct:
            cuts.append(i)
        for i in range(2, (actual_distinct_len+1)):
            if i == bins:
                lowercut = cuts[i-2]
                uppercut = float('Inf')
            else:
                lowercut = cuts[i-2]
                uppercut = cuts[i-1]
            actual_cnt = ((actual >= lowercut) & (actual < uppercut)).sum()+1
            predict_cnt = ((predict >= lowercut) & (predict < uppercut)).sum()+1
            actual_pct = (actual_cnt+0.0) / actual_len
            predict_pct = (predict_cnt+0.0) / predict_len
            psi_cut.append((actual_pct-predict_pct) * math.log(actual_pct/predict_pct))
            actual_bins.append(actual_pct)
            predict_bins.append(predict_pct)
    else:
        for i in range(1, bins):
            cuts.append(actual_min+i*binlen)
        for i in range(1, (bins+1)):
            if i == 1:
                lowercut = float('-Inf')
                uppercut = cuts[i-1]
            elif i == bins:
                lowercut = cuts[i-2]
                uppercut = float('Inf')
            else:
                lowercut = cuts[i-2]
                uppercut = cuts[i-1]
            actual_cnt = ((actual >= lowercut) & (actual < uppercut)).sum()+1
            predict_cnt = ((predict >= lowercut) & (predict < uppercut)).sum()+1
            actual_pct = (actual_cnt+0.0) / actual_len
            predict_pct = (predict_cnt+0.0) / predict_len
            psi_cut.append((actual_pct-predict_pct) * math.log(actual_pct/predict_pct))
            actual_bins.append(actual_pct)
            predict_bins.append(predict_pct)
    psi = sum(psi_cut)
    nbins = len(actual_bins)
    xlab = np.arange(1, nbins+1)
    psi_dict['psi'] = psi
    return psi_dict



def gen_label(threshold1, threshold2, x):
    if int(x) <= threshold1:
        return 0
    elif int(x) >= threshold2:
        return 1
    else: 
        return -1


In [33]:
# xgb

In [29]:
from sklearn.linear_model import LogisticRegression
import pandas as pd 
import numpy as np 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sys
import math
from sklearn.metrics import roc_auc_score
import warnings
import math
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV
import gc
import xgboost as xgb

In [30]:
train_label = baseline_feature.loc[train_index, 'label']
test_label = baseline_feature.loc[test_index, 'label']

In [31]:
D_train = xgb.DMatrix(stat_feature_train, label=train_label)
D_test = xgb.DMatrix(stat_feature_test, label=test_label)

In [32]:
ntrees = 2000
param={'max_depth':3, 'eta':0.07, 'silent':1, 'objective':'binary:logistic', 'eval_metric':'auc', 'scale_pos_weight':1,
       'subsample':0.75, 'seed':41,'colsample_bytree':0.75, 'lambda':1, 'alpha':1 ,'base_score':0.5, 'verbose_eval':10,
      }

watchlist = [(D_train,'train'), (D_test, 'test')]
#'min_child_weight': 100
bst = xgb.train(param, D_train, ntrees, watchlist, early_stopping_rounds=50)

[0]	train-auc:0.737256	test-auc:0.735703
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 50 rounds.
[1]	train-auc:0.753104	test-auc:0.750417
[2]	train-auc:0.758241	test-auc:0.755771
[3]	train-auc:0.767807	test-auc:0.767887
[4]	train-auc:0.770757	test-auc:0.771242
[5]	train-auc:0.778302	test-auc:0.777951
[6]	train-auc:0.781024	test-auc:0.780451
[7]	train-auc:0.781204	test-auc:0.780813
[8]	train-auc:0.781117	test-auc:0.780756
[9]	train-auc:0.78863	test-auc:0.786168
[10]	train-auc:0.789173	test-auc:0.786566
[11]	train-auc:0.789496	test-auc:0.787012
[12]	train-auc:0.793926	test-auc:0.790976
[13]	train-auc:0.793672	test-auc:0.790855
[14]	train-auc:0.794125	test-auc:0.791341
[15]	train-auc:0.796661	test-auc:0.793833
[16]	train-auc:0.796348	test-auc:0.793629
[17]	train-auc:0.797035	test-auc:0.794354
[18]	train-auc:0.797941	test-auc:0.795285
[19]	train-auc:0.798479	test-auc:0.795672
[20]	train-auc:0.799254	test-a